## Cryptocurrency trading bot testing file.

### Install if needed.

In [ ]:
##%pip install python-binance==0.7.5
#%pip install python-binance

### Examples loading directly from files.

In [ ]:
import pandas as pd

crypto_exchange_info = 'crypto_logs/crypto_exchange_info.txt'
exchange_info = pd.read_csv(crypto_exchange_info, header=0, index_col=0)
exchange_info

In [ ]:
import pandas as pd
crypto_output_log_15s = 'crypto_logs/crypto_output_log_15s.txt'
df_15s = pd.read_csv(crypto_output_log_15s, header=[0, 1], index_col=0)
df_15s.index = pd.DatetimeIndex(df_15s.index)
df_15s

### Load from logged files.

In [ ]:
from crypto_logger import init_loggers
crypto_loggers = init_loggers()
crypto_loggers

In [ ]:
input_15s = crypto_loggers['input_15s']
output_15s = crypto_loggers['output_15s']
output_1min = crypto_loggers['output_1min']
output_30min = crypto_loggers['output_30min']
output_1h = crypto_loggers['output_1h']
output_1d = crypto_loggers['output_1d']

In [ ]:
df_input_15s = input_15s.maybe_get_from_file(dataset=None, inputs=False, screened=False)
df_input_15s

In [ ]:
df_output_15s = output_15s.maybe_get_from_file(dataset=None, inputs=False, screened=False)
df_output_15s

In [ ]:
df_output_1min = output_1min.maybe_get_from_file(dataset=None, inputs=False, screened=False)
df_output_1min

In [ ]:
df_output_30min = output_30min.maybe_get_from_file(dataset=None, inputs=False, screened=False)
df_output_30min

In [ ]:
df_output_1h = output_1h.maybe_get_from_file(dataset=None, inputs=False, screened=False)
df_output_1h

In [ ]:
df_output_1d = output_1d.maybe_get_from_file(dataset=None, inputs=False, screened=False)
df_output_1d

In [ ]:
df_input_15s_screened = input_15s.maybe_get_from_file(dataset=None, inputs=False, screened=True)
df_input_15s_screened

In [ ]:
df_output_15s_screened = output_15s.maybe_get_from_file(dataset=None, inputs=False, screened=True)
df_output_15s_screened

In [ ]:
df_output_1min_screened = output_1min.maybe_get_from_file(dataset=None, inputs=False, screened=True)
df_output_1min_screened

In [ ]:
df_output_30min_screened = output_30min.maybe_get_from_file(dataset=None, inputs=False, screened=True)
df_output_30min_screened

In [ ]:
df_output_1h_screened = output_1h.maybe_get_from_file(dataset=None, inputs=False, screened=True)
df_output_1h_screened

In [ ]:
df_output_1d_screened = output_1d.maybe_get_from_file(dataset=None, inputs=False, screened=True)
df_output_1d_screened

In [ ]:
df = df_output_1d.copy()
df

In [ ]:
df['BTC']

### Uncomment to start logger.

In [ ]:
#from crypto_logger import loop_loggers
#loop_loggers(crypto_loggers)

### Test indicators individually.

In [ ]:
import pandas_ta as ta

def filter_in_market(function, dataset):
    def f(x):
        x = x.loc[:,~x.columns.duplicated()]
        return function(x)
    tickers_list = dataset.columns.get_level_values(0).unique().tolist()
    return pd.Series([ticker for ticker in tickers_list if f(dataset[ticker])], dtype='str')

def get_relative_volume_levels_smoothed_trigger(data, average1=26, average2=14):
    volume = data['volume']
    volume_average = ta.sma(close=volume, length=average1)
    relative_volume = volume / volume_average
    smoothed_relative_volume = ta.sma(close=relative_volume, length=average2)
    return (smoothed_relative_volume > threshold).iat[-1]

def get_relative_volume_levels_at_time_smoothed_thresholded(data):
    try:
        volume = data['volume']
        #volume = volume.groupby(pd.Grouper(freq='D')).cumsum()
        cum_volume = volume.groupby(pd.Grouper(freq='24h')).cumsum()
        #volume = volume.groupby(pd.Grouper(freq='60m')).cumsum()
        cum_rvol = (cum_volume / cum_volume.shift(1)).fillna(method='pad')
        rvol = (volume / volume.shift(1)).fillna(method='pad')
        bar_up = (data['close'] > data['open'])
        bar_up |= (data['close'] == data['open']) & (data['close'].diff() > 0)
        bar_up = bar_up.astype(int)
        bar_up = bar_up * 2 - 1
        #rvol *= bar_up
        cum_rvol_dir = cum_rvol * bar_up
        rvol_dir = rvol * bar_up
        rvol_indicator = ta.hma(rvol, length=14, talib=True)
        rvol_dir_indicator = ta.hma(rvol_dir, length=14, talib=True)
        cum_rvol_indicator = ta.hma(cum_rvol, length=14, talib=True)
        cum_rvol_dir_indicator = ta.hma(cum_rvol_dir, length=14, talib=True)
        rvol_indicator = rvol_indicator.rename('relative_volume_levels_smoothed')
        rvol_dir_indicator = rvol_dir_indicator.rename('relative_volume_levels_dir_smoothed')
        cum_rvol_indicator = cum_rvol_indicator.rename('cum_relative_volume_levels_smoothed')
        cum_rvol_dir_indicator = cum_rvol_dir_indicator.rename('cum_relative_volume_levels_dir_smoothed')
        #threshold = (ta.sma(rvol, length=100, talib=True) + ta.stdev(rvol, length=100, talib=True))
        threshold_dir = 0
        threshold = 2
        rvol_thresholded = (rvol_indicator > threshold).iat[-1]
        rvol_dir_thresholded = (rvol_dir_indicator > threshold_dir).iat[-1]
        cum_rvol_thresholded = (cum_rvol_indicator > threshold).iat[-1]
        cum_rvol_dir_thresholded = (cum_rvol_dir_indicator > threshold_dir).iat[-1]
        trigger = (rvol_thresholded | rvol_dir_thresholded | cum_rvol_thresholded | cum_rvol_dir_thresholded)
    except Exception as e:
        print('rvol exception:', e)
        trigger = False
    return trigger

def get_positive_trend_strength_trigger(data):
    ADX = data.ta.adx(talib=True)
    return (ADX['ADX_14'] < 0.20).iloc[-3] & (ADX['ADX_14'] > 0.20).iat[-2]

def get_not_negative_trend_strength_trigger(data):
    ADX = data.ta.adx(length=14, lensig=8, talib=True)
    return ((ADX['DMP_14'] > ADX['DMN_14']) & (ADX['ADX_14'] > 0.30)).iat[-1]

def get_not_negative_rebound_trigger(data):
    CCI = data.ta.cci(length=22, talib=True)
    MFI = data.ta.mfi(length=11, talib=True)
    return ((CCI > 0) | (MFI > 20)).iat[-1]

def get_positive_choppiness_trigger(data):
    CHOP = data.ta.chop(talib=True)
    return CHOP.iat[-1] < 38.2

def get_positive_phase_trigger(data):
    MACD = data.ta.macd(talib=True)
    histogram = MACD['MACDs_12_26_9'] - MACD['MACD_12_26_9']
    return ((histogram > histogram.shift(1)) | \
            (MACD['MACD_12_26_9'] > MACD['MACDs_12_26_9'])).iat[-1]

def get_positive_phase_trigger(data):
    MACD = data.ta.macd(talib=True)
    histogram = MACD['MACDs_12_26_9'] - MACD['MACD_12_26_9']
    return ((histogram.iloc[-2] > histogram.iat[-2]) or \
            (MACD['MACD_12_26_9'].iat[-1] > MACD['MACDs_12_26_9'].iat[-1]))

def get_not_square_wave_triggers(data, multiplier_schedule):
    triggers = True
    for multiplier in multiplier_schedule:
        period_1 = -4 * multiplier
        uniques_1 = 2 * multiplier
        square_wave_trigger_1 = (data.iloc[period_1:]['close'].unique().size < uniques_1)
        if square_wave_trigger_1:
            triggers = False
            break
        else:
            period_2 = -15 * multiplier
            uniques_2 = 6 * multiplier
            square_wave_trigger_2 = (data.iloc[period_2:]['close'].unique().size < uniques_2)
            if square_wave_trigger_2:
                triggers = False
                break
    return triggers

def get_minute_not_square_wave_triggers(data):
    return get_not_square_wave_triggers(data, multiplier_schedule=[1, 2, 3, 5, 10, 15, 20, 45])

def get_hourly_not_square_wave_triggers(data):
    return get_not_square_wave_triggers(data, multiplier_schedule=[1])

def get_daily_not_square_wave_triggers(data):
    return get_not_square_wave_triggers(data, multiplier_schedule=[1])

def get_daily_volume_minimum_trigger(data):
    return (data['volume'] > 1000000).iat[-1]

def get_daily_volume_change_trigger(data):
    return ((data['volume'].pct_change(1) * 100) > 300).iat[-1]

def get_minute_daily_volume_minimum_trigger(data):
    return (data['rolling_base_volume'] > 1000000).iat[-1]

def get_minute_daily_volume_change_trigger(data):
    return ((data['rolling_base_volume'].pct_change(1440) * 100) > 300).iat[-1]

def get_rising_volume_trigger(data):
    return (data['rolling_base_volume'].diff(1) > 0).iat[-1]

In [ ]:
df = df_1d.copy()
df.columns = df.columns.swaplevel(0, 1)
df = df.rename(columns={'base_volume': 'volume'})
df.columns = df.columns.swaplevel(0, 1)
filtered_1 = set(filter_in_market(get_daily_not_square_wave_triggers, df).tolist())
filtered_2 = set(filter_in_market(get_daily_volume_minimum_trigger, df).tolist())
filtered_3 = set(filter_in_market(get_relative_volume_levels_smoothed_trigger, df).tolist())
filtered_1d = pd.Series(list(filtered_1 & filtered_2 & filtered_3))
filtered_1d

In [ ]:
df = df_1h.copy()
df.columns = df.columns.swaplevel(0, 1)
df = df.rename(columns={'base_volume': 'volume'})
df.columns = df.columns.swaplevel(0, 1)
filtered_1 = set(filter_in_market(get_hourly_not_square_wave_triggers, df).tolist())
filtered_2 = set(filter_in_market(get_relative_volume_levels_at_time_smoothed_thresholded, df).tolist())
filtered_1h = pd.Series(list(filtered_1 & filtered_2))
filtered_1h

In [ ]:
from cryptocurrency.indicators import filter_in_market, screen_one
df = df_1min.copy()
df.columns = df.columns.swaplevel(0, 1)
df = df.rename(columns={'base_volume': 'volume'})
df.columns = df.columns.swaplevel(0, 1)
filtered = filter_in_market(screen_one, df).tolist()
filtered

In [ ]:
df = df_1min.copy()
df.columns = df.columns.swaplevel(0, 1)
df = df.rename(columns={'base_volume': 'volume'})
df.columns = df.columns.swaplevel(0, 1)
filtered_1 = set(filter_in_market(get_minute_not_square_wave_triggers, df).tolist())
filtered_2 = set(filter_in_market(get_minute_daily_volume_minimum_trigger, df).tolist())
filtered_3 = set(filter_in_market(get_minute_daily_volume_change_trigger, df).tolist())
filtered_4 = set(filter_in_market(get_rising_volume_trigger, df).tolist())
filtered_1min = pd.Series(list(filtered_1 & filtered_2 & filtered_3 & filtered_4))
filtered_1min

In [ ]:
list(set(filtered_1h.tolist()) & set(filtered_1d.tolist()))

In [ ]:
list(set(filtered_1min.tolist()) & set(filtered_1h.tolist()))

In [ ]:
list(set(filtered_1min.tolist()) & set(filtered_1d.tolist()))

In [ ]:
list(set(filtered_1min.tolist()) & set(filtered_1h.tolist()) & set(filtered_1d.tolist()))

### Example testing conversion_table directly.

In [1]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion import get_timezone_offset_in_seconds
#from cryptocurrency.conversion_table import get_conversion_table

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info
offset_s = get_timezone_offset_in_seconds()

#conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, 
#                                        offset_s=offset_s, as_pair=False)
#conversion_table.sort_values(by='rolling_base_volume', ascending=False).reset_index(drop=True).head(50)

In [50]:
from cryptocurrency.conversion import convert_price
from cryptocurrency.conversion import get_base_asset_from_pair, get_quote_asset_from_pair
from cryptocurrency.conversion import get_shortest_pair_path_between_assets
import datetime
import pandas as pd

def get_conversion_table_from_binance(client, exchange_info, offset_s=0, dump_raw=False):
    conversion_table = pd.DataFrame(client.get_ticker())
    conversion_table = conversion_table[conversion_table['symbol'].isin(exchange_info['symbol'])]

    conversion_table['base_asset'] = \
        conversion_table['symbol'].apply(lambda x: get_base_asset_from_pair(x, exchange_info=exchange_info))
    conversion_table['quote_asset'] = \
        conversion_table['symbol'].apply(lambda x: get_quote_asset_from_pair(x, exchange_info=exchange_info))

    conversion_table = \
        conversion_table.rename(
            columns={'openPrice': 'open', 'highPrice': 'high', 'lowPrice': 'low', 'lastPrice': 'close', 
                     'lastQty': 'last_volume', 'volume': 'rolling_base_volume', 'quoteVolume': 'rolling_quote_volume', 
                     'bidPrice': 'bid_price', 'askPrice': 'ask_price', 'bidQty': 'bid_volume', 'askQty': 'ask_volume', 
                     'firstId': 'first_ID', 'lastId': 'last_ID', 'openTime': 'open_time', 'closeTime': 'date', 
                     'prevClosePrice': 'close_shifted', 'weightedAvgPrice': 'weighted_average_price', 
                     'priceChange': 'price_change', 'priceChangePercent': 'price_change_percent'})

    if dump_raw:
        conversion_table.to_csv('crypto_logs/conversion_table.txt')

    conversion_table[['open', 'high', 'low', 'close', 'close_shifted', 'rolling_base_volume', 
                      'rolling_quote_volume', 'bid_price', 'ask_price', 'bid_volume', 'ask_volume', 
                      'price_change_percent']] = \
        conversion_table[['open', 'high', 'low', 'close', 'close_shifted', 'rolling_base_volume', 
                          'rolling_quote_volume', 'bid_price', 'ask_price', 'bid_volume', 'ask_volume', 
                          'price_change_percent']].astype(float)

    conversion_table[['date', 'count']] = \
        conversion_table[['date', 'count']].astype(int)
    conversion_table['date'] = (conversion_table['date'] + offset_s * 1000)
    conversion_table['date'] /= 1000
    conversion_table['date'] = conversion_table['date'].apply(datetime.datetime.fromtimestamp)
    conversion_table['date'] = pd.DatetimeIndex(conversion_table['date'])
    return conversion_table.sort_values(by='date')

def process_conversion_table(conversion_table, exchange_info, as_pair=False, minimal=True, 
                             extra_minimal=True, super_extra_minimal=True, convert_to_USDT=True):
    """
    Fetches and prepares data used to calculate prices, volumes and other stats.
    :param client: object from python-binance useful for calling client.get_ticker().
    :param exchange_info: Pre-calculated exchange information on all tickers.
    :return: pd.DataFrame containing all preprocessed conversion table info.
    :column is_shorted: is the symbol made from inversion.
    :column symbol: concatenated string made from base_asset and quote_asset.
    :column shorted_symbol: symbol with inverted base_asset and quote_asset.
    :column base_asset: asset on the left.
    :column quote_asset: asset on the right.
    :column price_change: (close - open).
    :column price_change_percent: (((close - open) / open) * 100).
    :column USDT_price_change: (USDT_close - USDT_open).
    :column USDT_price_change_percent: (((USDT_close - USDT_open) / USDT_open) * 100).
    :column weighted_average_price: weighted average price.
    :column close_shifted: close price of the previous day.
    :column open: open price of the day.
    :column high: high price of the day.
    :column low: low price of the day.
    :column close: close price of the day.
    :column last_volume: volume of the last price update.
    :column USDT_bid_price: USDT-converted bid price.
    :column USDT_ask_price: USDT-converted ask price.
    :column USDT_bid_volume: USDT-converted bid volume.
    :column USDT_ask_volume: USDT-converted ask volume.
    :column bid_price: price of the bid.
    :column bid_volume: volume of the bid at bid_price.
    :column ask_price: price of the ask.
    :column ask_volume: volume of the ask at ask_price.
    :column rolling_base_volume: rolling_base_volume given by the API.
    :column rolling_quote_volume: rolling_quote_volume given by the API.
    :column open_time: date minus 24 hours.
    :column date: time from epoch in milliseconds of the last price update.
    :column first_ID: transaction ID from 1 day ago.
    :column last_ID: latest transaction ID.
    :column count: value calculated by subtracting first_ID from last_ID.
    :column USDT_open: USDT-converted open price.
    :column USDT_high: USDT-converted high price.
    :column USDT_low: USDT-converted low price.
    :column USDT_price: USDT-converted close price.
    :column rolling_USDT_base_volume: USDT-converted rolling_base_volume.
    :column rolling_USDT_quote_volume: USDT-converted rolling_quote_volume.
    :column rolling_traded_volume: sum by base_asset of all USDT-converted volumes.
    :column importance: rolling_USDT_base_volume divided by rolling_traded_volume.
    :column traded_price: sum by base_asset of all (close prices times importance).
    :column traded_bid_price: sum by base_asset of all (bid prices times importance).
    :column traded_ask_price: sum by base_asset of all (ask prices times importance).
    :column bid_ask_percent_change: ((ask_price - bid_price) / ask_price) * 100).
    :column bid_ask_volume_percent_change: ((bid_volume / (bid_volume + ask_volume)) * 100).
    :column traded_bid_ask_percent_change: ((traded_ask_price - traded_bid_price) / traded_ask_price) * 100).
    :column traded_bid_ask_volume_percent_change: ((traded_bid_volume / (traded_bid_volume + traded_ask_volume)) * 100).
    """
    if as_pair:
        super_extra_minimal = False
    else:
        convert_to_USDT = True

    if super_extra_minimal:
        extra_minimal = True

    if extra_minimal:
        minimal = True

    if minimal:
        conversion_table = \
            conversion_table[['symbol', 'base_asset', 'quote_asset', 'open', 'high', 'low', 'close', 
                              'rolling_base_volume', 'rolling_quote_volume', 'count', 'bid_price', 
                              'ask_price', 'bid_volume', 'ask_volume', 'date', 'price_change_percent']]

    conversion_table = conversion_table.copy()
    conversion_table['rolling_base_quote_volume'] = \
        conversion_table['rolling_quote_volume'] / conversion_table['close']

    if convert_to_USDT:
        conversion_table['shortest_path'] = \
            conversion_table.apply(lambda x: get_shortest_pair_path_between_assets(
                from_asset=x['base_asset'], to_asset='USDT', exchange_info=exchange_info, 
                priority='accuracy'), axis='columns')
        if not extra_minimal:
            conversion_table['high_pre_conversion'] = \
                (((conversion_table['high'] - conversion_table['close']) / conversion_table['close']) + 1)
            conversion_table['low_pre_conversion'] = \
                (((conversion_table['low'] - conversion_table['close']) / conversion_table['close']) + 1)
        if not super_extra_minimal:
            conversion_table['bid_pre_conversion'] = \
                (((conversion_table['bid_price'] - conversion_table['close']) / conversion_table['close']) + 1)
            conversion_table['ask_pre_conversion'] = \
                (((conversion_table['ask_price'] - conversion_table['close']) / conversion_table['close']) + 1)
            conversion_table['USDT_open'] = \
                conversion_table.apply(
                    lambda x: convert_price(
                        size=1, from_asset=x['base_asset'], 
                        to_asset='USDT', 
                        conversion_table=conversion_table, 
                        exchange_info=exchange_info, 
                        shortest_path=x['shortest_path'], 
                        key='open', priority='accuracy'), axis='columns')
        conversion_table['USDT_price'] = \
            conversion_table.apply(
                lambda x: convert_price(
                    size=1, from_asset=x['base_asset'], 
                    to_asset='USDT', 
                    conversion_table=conversion_table, 
                    exchange_info=exchange_info, 
                    shortest_path=x['shortest_path'], 
                    key='close', priority='accuracy'), axis='columns')
        if not extra_minimal:
            conversion_table['USDT_high'] = \
                conversion_table['USDT_price'].astype(float) * conversion_table['high_pre_conversion']
            conversion_table['USDT_low'] = \
                conversion_table['USDT_price'].astype(float) * conversion_table['low_pre_conversion']
        if not super_extra_minimal:
            conversion_table['USDT_bid_price'] = \
                conversion_table['USDT_price'].astype(float) * conversion_table['bid_pre_conversion']
            conversion_table['USDT_ask_price'] = \
                conversion_table['USDT_price'].astype(float) * conversion_table['ask_pre_conversion']
            conversion_table['USDT_bid_volume'] = \
                conversion_table['bid_volume'] * conversion_table['USDT_bid_price'].astype(float)
            conversion_table['USDT_ask_volume'] = \
                conversion_table['ask_volume'] * conversion_table['USDT_ask_price'].astype(float)

        conversion_table['rolling_USDT_base_volume'] = \
            conversion_table['rolling_base_volume'] * conversion_table['USDT_price'].astype(float)
        conversion_table['rolling_USDT_quote_volume'] = \
            conversion_table['rolling_base_quote_volume'] * conversion_table['USDT_price'].astype(float)
        conversion_table = conversion_table.drop(columns=['rolling_base_quote_volume'])

        if super_extra_minimal:
            price_change_percent = conversion_table[['base_asset', 'price_change_percent']]
            price_change_percent = \
                price_change_percent.groupby(by='base_asset').agg(lambda x: x.iloc[x.abs().argmax()])
            price_change_percent = price_change_percent['price_change_percent']
            conversion_table['price_change_percent'] = \
                conversion_table.apply(lambda x: price_change_percent.loc[x['base_asset']], axis='columns')
        else:
            conversion_table['USDT_price_change'] = \
                (conversion_table['USDT_price'].astype(float) - conversion_table['USDT_open'].astype(float))
            conversion_table['USDT_price_change_percent'] = \
                ((conversion_table['USDT_price_change'] / conversion_table['USDT_open'].astype(float)) * 100)

        conversion_table['is_shorted'] = False

        conversion_table_swapped = conversion_table.copy()
        if minimal:
            if extra_minimal:
                if super_extra_minimal:
                    conversion_table_swapped.loc[:, ['symbol', 'price_change_percent', 'close', 'ask_price', 
                                                     'ask_volume', 'bid_price', 'bid_volume', 'open', 
                                                     'rolling_quote_volume', 'rolling_base_volume', 'date', 
                                                     'count', 'quote_asset', 'base_asset', 'USDT_price', 
                                                     'rolling_USDT_quote_volume', 'rolling_USDT_base_volume', 
                                                     'is_shorted']] = \
                        conversion_table_swapped.loc[:, ['symbol', 'price_change_percent', 'close', 'bid_price', 
                                                         'bid_volume', 'ask_price', 'ask_volume', 'open', 
                                                         'rolling_base_volume', 'rolling_quote_volume', 
                                                         'date', 'count', 'base_asset', 'quote_asset', 
                                                         'USDT_price', 'rolling_USDT_base_volume', 
                                                         'rolling_USDT_quote_volume', 'is_shorted']].values
                else:
                    conversion_table_swapped.loc[:, ['symbol', 'price_change_percent', 'close', 'ask_price', 
                                                     'ask_volume', 'bid_price', 'bid_volume', 'open', 
                                                     'rolling_quote_volume', 'rolling_base_volume', 'date', 
                                                     'count', 'quote_asset', 'base_asset', 'USDT_open', 
                                                     'USDT_price', 'rolling_USDT_quote_volume', 
                                                     'rolling_USDT_base_volume', 'USDT_ask_price', 
                                                     'USDT_bid_price', 'USDT_ask_volume', 'USDT_bid_volume', 
                                                     'USDT_price_change_percent', 'is_shorted']] = \
                        conversion_table_swapped.loc[:, ['symbol', 'price_change_percent', 'close', 'bid_price', 
                                                         'bid_volume', 'ask_price', 'ask_volume', 'open', 
                                                         'rolling_base_volume', 'rolling_quote_volume', 'date', 
                                                         'count', 'base_asset', 'quote_asset', 'USDT_open', 
                                                         'USDT_price', 'rolling_USDT_base_volume', 
                                                         'rolling_USDT_quote_volume', 'USDT_bid_price', 
                                                         'USDT_ask_price', 'USDT_bid_volume', 'USDT_ask_volume', 
                                                         'USDT_price_change_percent', 'is_shorted']].values
            else:
                conversion_table_swapped.loc[:, ['symbol', 'price_change_percent', 'close', 'ask_price', 
                                                 'ask_volume', 'bid_price', 'bid_volume', 'open', 
                                                 'rolling_quote_volume', 'rolling_base_volume', 'date', 
                                                 'count', 'quote_asset', 'base_asset', 'USDT_open', 'USDT_price', 
                                                 'rolling_USDT_quote_volume', 'rolling_USDT_base_volume', 
                                                 'USDT_ask_price', 'USDT_bid_price', 'USDT_ask_volume', 
                                                 'USDT_bid_volume', 'USDT_price_change_percent', 'is_shorted']] = \
                    conversion_table_swapped.loc[:, ['symbol', 'price_change_percent', 'close', 'bid_price', 
                                                     'bid_volume', 'ask_price', 'ask_volume', 'open', 
                                                     'rolling_base_volume', 'rolling_quote_volume', 'date', 
                                                     'count', 'base_asset', 'quote_asset', 'USDT_open', 'USDT_price', 
                                                     'rolling_USDT_base_volume', 'rolling_USDT_quote_volume', 
                                                     'USDT_bid_price', 'USDT_ask_price', 'USDT_bid_volume', 
                                                     'USDT_ask_volume', 'USDT_price_change_percent', 
                                                     'is_shorted']].values
        else:
            conversion_table_swapped.loc[:, ['symbol', 'price_change', 'price_change_percent', 
                                             'weighted_average_price', 'close_shifted', 'close', 
                                             'last_volume', 'ask_price', 'ask_volume', 'bid_price', 
                                             'bid_volume', 'open', 'high', 'low', 'rolling_quote_volume', 
                                             'rolling_base_volume', 'open_time', 'date', 'first_ID', 
                                             'last_ID', 'count', 'quote_asset', 'base_asset', 'USDT_open', 
                                             'USDT_high', 'USDT_low', 'USDT_price', 'rolling_USDT_quote_volume', 
                                             'rolling_USDT_base_volume', 'USDT_ask_price', 'USDT_bid_price', 
                                             'USDT_ask_volume', 'USDT_bid_volume', 'USDT_price_change', 
                                             'USDT_price_change_percent', 'is_shorted']] = \
                conversion_table_swapped.loc[:, ['symbol', 'price_change', 'price_change_percent', 
                                                 'weighted_average_price', 'close_shifted', 'close', 
                                                 'last_volume', 'bid_price', 'bid_volume', 'ask_price', 
                                                 'ask_volume', 'open', 'high', 'low', 'rolling_base_volume', 
                                                 'rolling_quote_volume', 'open_time', 'date', 'first_ID', 
                                                 'last_ID', 'count', 'base_asset', 'quote_asset', 
                                                 'USDT_open', 'USDT_high', 'USDT_low', 'USDT_price', 
                                                 'rolling_USDT_base_volume', 'rolling_USDT_quote_volume', 
                                                 'USDT_bid_price', 'USDT_ask_price', 'USDT_bid_volume', 
                                                 'USDT_ask_volume', 'USDT_price_change', 
                                                 'USDT_price_change_percent', 'is_shorted']].values

        conversion_table_swapped['symbol'] = \
            conversion_table_swapped['base_asset'] + conversion_table_swapped['quote_asset']

        if minimal:
            if extra_minimal:
                if super_extra_minimal:
                    conversion_table_swapped.loc[:, ['open', 'close', 'bid_price', 'ask_price', 'USDT_price']] = \
                        1 / conversion_table_swapped.loc[:, ['open', 'close', 'bid_price', 'ask_price', 
                                                             'USDT_price']].astype(float)
                else:
                    conversion_table_swapped.loc[:, ['open', 'close', 'bid_price', 'ask_price', 'USDT_open', 
                                                     'USDT_price', 'USDT_price_change_percent']] = \
                        1 / conversion_table_swapped.loc[:, ['open', 'close', 'bid_price', 'ask_price', 
                                                             'USDT_open', 'USDT_price', 
                                                             'USDT_price_change_percent']].astype(float)
            else:
                conversion_table_swapped.loc[:, ['open', 'close', 'bid_price', 'ask_price', 'USDT_open', 
                                                 'USDT_price', 'USDT_bid_price', 'USDT_ask_price', 
                                                 'USDT_price_change_percent']] = \
                    1 / conversion_table_swapped.loc[:, ['open', 'close', 'bid_price', 'ask_price', 'USDT_open', 
                                                         'USDT_price', 'USDT_bid_price', 'USDT_ask_price', 
                                                         'USDT_price_change_percent']].astype(float)
        else:
            conversion_table_swapped.loc[:, ['open', 'high', 'low', 'close', 'close_shifted', 'bid_price', 
                                             'ask_price', 'USDT_open', 'USDT_high', 'USDT_low', 'USDT_price', 
                                             'USDT_bid_price', 'USDT_ask_price', 'USDT_price_change', 
                                             'USDT_price_change_percent']] = \
                1 / conversion_table_swapped.loc[:, ['open', 'high', 'low', 'close', 'close_shifted', 
                                                     'bid_price', 'ask_price', 'USDT_open', 'USDT_high', 
                                                     'USDT_low', 'USDT_price', 'USDT_bid_price', 'USDT_ask_price', 
                                                     'USDT_price_change', 'USDT_price_change_percent']].astype(float)
        conversion_table_swapped['is_shorted'] = True

        conversion_table = pd.concat([conversion_table, conversion_table_swapped], join='outer', axis='index')

        traded_volume = conversion_table.groupby(by='base_asset').agg('sum')
        traded_volume = traded_volume['rolling_USDT_base_volume']
        conversion_table['rolling_traded_volume'] = \
            conversion_table.apply(lambda x: traded_volume.loc[x['base_asset']], axis='columns')
        if not extra_minimal:
            traded_bid_volume = conversion_table.groupby(by='base_asset').agg('sum')
            traded_bid_volume = traded_bid_volume['USDT_bid_volume']
            conversion_table['traded_bid_volume'] = \
                conversion_table.apply(lambda x: traded_bid_volume.loc[x['base_asset']], axis='columns')
            traded_ask_volume = conversion_table.groupby(by='base_asset').agg('sum')
            traded_ask_volume = traded_ask_volume['USDT_ask_volume']
            conversion_table['traded_ask_volume'] = \
                conversion_table.apply(lambda x: traded_ask_volume.loc[x['base_asset']], axis='columns')

        conversion_table['importance'] = \
            conversion_table['rolling_USDT_base_volume'] / conversion_table['rolling_traded_volume']
        conversion_table['importance_weighted_price'] = \
            conversion_table['USDT_price'].astype(float) * conversion_table['importance']

        if not extra_minimal:
            conversion_table['importance_weighted_bid_price'] = \
                conversion_table['USDT_bid_price'].astype(float) * conversion_table['importance']
            conversion_table['importance_weighted_ask_price'] = \
                conversion_table['USDT_ask_price'].astype(float) * conversion_table['importance']

        importance_weighted_price = conversion_table.groupby(by='base_asset').agg('sum')
        importance_weighted_price = importance_weighted_price['importance_weighted_price']
        conversion_table['traded_price'] = \
            conversion_table.apply(lambda x: importance_weighted_price.loc[x['base_asset']], axis='columns')

        if not extra_minimal:
            importance_weighted_bid_price = conversion_table.groupby(by='base_asset').agg('sum')
            importance_weighted_bid_price = importance_weighted_bid_price['importance_weighted_bid_price']
            conversion_table['traded_bid_price'] = \
                conversion_table.apply(lambda x: importance_weighted_bid_price.loc[x['base_asset']], axis='columns')
            importance_weighted_ask_price = conversion_table.groupby(by='base_asset').agg('sum')
            importance_weighted_ask_price = importance_weighted_ask_price['importance_weighted_ask_price']
            conversion_table['traded_ask_price'] = \
                conversion_table.apply(lambda x: importance_weighted_ask_price.loc[x['base_asset']], axis='columns')

            conversion_table['traded_bid_ask_percent_change'] = \
                ((conversion_table['traded_ask_price'] - conversion_table['traded_bid_price']) / \
                 conversion_table['traded_ask_price'])
            conversion_table['traded_bid_ask_volume_percent_change'] = \
                (conversion_table['traded_bid_volume'] / (conversion_table['traded_bid_volume'] + \
                                                          conversion_table['traded_ask_volume']))
            conversion_table[['traded_bid_ask_percent_change', 'traded_bid_ask_volume_percent_change']] *= 100

        conversion_table = conversion_table[~conversion_table['is_shorted']]

    conversion_table['bid_ask_percent_change'] = \
        ((conversion_table['ask_price'] - conversion_table['bid_price']) / conversion_table['ask_price'])
    conversion_table['bid_ask_volume_percent_change'] = \
        (conversion_table['bid_volume'] / (conversion_table['bid_volume'] + conversion_table['ask_volume']))
    conversion_table[['bid_ask_percent_change', 'bid_ask_volume_percent_change']] *= 100
    if as_pair and convert_to_USDT:
        if minimal:
            if extra_minimal:
                conversion_table = \
                    conversion_table[['symbol', 'base_asset', 'quote_asset', 'price_change_percent', 'close', 
                                      'bid_price', 'bid_volume', 'ask_price', 'ask_volume', 'date', 
                                      'count', 'rolling_base_volume', 'rolling_quote_volume', 
                                      'USDT_price_change_percent', 'USDT_price', 'rolling_USDT_base_volume', 
                                      'rolling_USDT_quote_volume', 'rolling_traded_volume', 'traded_price', 
                                      'bid_ask_percent_change', 'bid_ask_volume_percent_change']]
            else:
                conversion_table = \
                    conversion_table[['symbol', 'base_asset', 'quote_asset', 'price_change_percent', 'close', 
                                      'bid_price', 'bid_volume', 'ask_price', 'ask_volume', 'date', 
                                      'count', 'rolling_base_volume', 'rolling_quote_volume', 
                                      'USDT_price_change_percent', 'USDT_price', 'rolling_USDT_base_volume', 
                                      'rolling_USDT_quote_volume', 'USDT_bid_price', 'USDT_ask_price', 
                                      'USDT_bid_volume', 'USDT_ask_volume', 'rolling_traded_volume', 
                                      'traded_bid_volume', 'traded_ask_volume', 'traded_price', 
                                      'traded_bid_price', 'traded_ask_price', 'bid_ask_percent_change', 
                                      'bid_ask_volume_percent_change', 'traded_bid_ask_percent_change', 
                                      'traded_bid_ask_volume_percent_change']]
        else:
            conversion_table = \
                conversion_table[['symbol', 'base_asset', 'quote_asset', 'is_shorted', 'price_change_percent', 
                                  'weighted_average_price', 'open', 'high', 'low', 'close', 'close_shifted', 
                                  'last_volume', 'bid_price', 'bid_volume', 'ask_price', 'ask_volume', 
                                  'date', 'last_ID', 'count', 'rolling_base_volume', 'rolling_quote_volume', 
                                  'importance', 'USDT_price_change_percent', 'USDT_open', 
                                  'USDT_high', 'USDT_low', 'USDT_price', 'rolling_USDT_base_volume', 
                                  'rolling_USDT_quote_volume', 'USDT_bid_price', 'USDT_ask_price', 
                                  'USDT_bid_volume', 'USDT_ask_volume', 'rolling_traded_volume', 
                                  'traded_bid_volume', 'traded_ask_volume', 'traded_price', 
                                  'traded_bid_price', 'traded_ask_price', 'bid_ask_percent_change', 
                                  'bid_ask_volume_percent_change', 'traded_bid_ask_percent_change', 
                                  'traded_bid_ask_volume_percent_change']]
    else:
        if convert_to_USDT:
            if minimal:
                if extra_minimal:
                    conversion_table = \
                        conversion_table[['base_asset', 'price_change_percent', 'date', 
                                          'bid_volume', 'ask_volume', 'bid_price', 'close', 'ask_price', 
                                          'count', 'rolling_traded_volume', 'bid_ask_percent_change', 
                                          'bid_ask_volume_percent_change', 'traded_price']]
                else:
                    conversion_table = \
                        conversion_table[['base_asset', 'USDT_price_change_percent', 'date', 'count', 
                                          'rolling_traded_volume', 'traded_bid_volume', 
                                          'traded_ask_volume', 'traded_price', 'traded_bid_price', 
                                          'traded_ask_price', 'traded_bid_ask_percent_change', 
                                          'traded_bid_ask_volume_percent_change']]
            else:
                conversion_table = \
                    conversion_table[['base_asset', 'USDT_price_change_percent', 'date', 'last_ID', 
                                      'count', 'rolling_traded_volume', 'traded_bid_volume', 
                                      'traded_ask_volume', 'traded_price', 'traded_bid_price', 
                                      'traded_ask_price', 'traded_bid_ask_percent_change', 
                                      'traded_bid_ask_volume_percent_change']]
            conversion_table['rolling_quote_volume'] = conversion_table['rolling_traded_volume'].copy()
            if 'close' in conversion_table.columns:
                conversion_table = conversion_table.drop(columns=['close'])
            if extra_minimal:
                conversion_table = \
                    conversion_table.rename(columns={'rolling_traded_volume': 'rolling_base_volume', 
                                                     'traded_price': 'close'})
                if not super_extra_minimal:
                    conversion_table = \
                        conversion_table.rename(columns={'USDT_price_change_percent': 'price_change_percent'})
            else:
                conversion_table = \
                    conversion_table.rename(columns={'USDT_price_change_percent': 'price_change_percent', 
                                                     'rolling_traded_volume': 'rolling_base_volume', 
                                                     'traded_bid_volume': 'bid_volume', 
                                                     'traded_ask_volume': 'ask_volume', 
                                                     'traded_price': 'close', 
                                                     'traded_bid_price': 'bid_price', 
                                                     'traded_ask_price': 'ask_price', 
                                                     'traded_bid_ask_percent_change': 'bid_ask_percent_change', 
                                                     'traded_bid_ask_volume_percent_change': 
                                                             'bid_ask_volume_percent_change'})
        if not as_pair:
            conversion_table['symbol'] = conversion_table['base_asset'].copy()
            conversion_table['quote_asset'] = conversion_table['base_asset'].copy()
            if minimal:
                #df = conversion_table.groupby(by=['base_asset']).agg({'date': 'max', 'count': 'sum'})
                df = conversion_table.groupby(by=['base_asset']).agg({'date': 'max', 'count': 'mean'})
                conversion_table.loc[:, ['date', 'count']] = \
                    conversion_table.apply(lambda x: df.loc[x['base_asset']], axis='columns')
                if super_extra_minimal:
                    df = conversion_table.groupby(by=['base_asset']).agg({'bid_ask_percent_change': 'min', 
                                                                          'bid_ask_volume_percent_change': 'max'})
                    conversion_table.loc[:, ['bid_ask_percent_change', 'bid_ask_volume_percent_change']] = \
                        conversion_table.apply(lambda x: df.loc[x['base_asset']], axis='columns')
            else:
                #df = conversion_table.groupby(by=['base_asset']).agg({'date': 'max', 'last_ID': 'sum', 'count': 'sum'})
                df = conversion_table.groupby(by=['base_asset']).agg({'date': 'max', 'last_ID': 'sum', 'count': 'mean'})
                conversion_table.loc[:, ['date', 'last_ID', 'count']] = \
                    conversion_table.apply(lambda x: df.loc[x['base_asset']], axis='columns')
            conversion_table = conversion_table.drop_duplicates(subset=['base_asset'], keep='first')
        conversion_table = conversion_table.reset_index(drop=True)
    return conversion_table.set_index('date').sort_index(axis='index')

def get_conversion_table(client, exchange_info, offset_s=0, dump_raw=False, as_pair=True, minimal=False, 
                         extra_minimal=False, super_extra_minimal=False, convert_to_USDT=False):
    conversion_table = get_conversion_table_from_binance(client=client, exchange_info=exchange_info, 
                                                         offset_s=offset_s, dump_raw=dump_raw)
    return process_conversion_table(conversion_table=conversion_table, exchange_info=exchange_info, 
                                    as_pair=as_pair, minimal=minimal, extra_minimal=extra_minimal, 
                                    super_extra_minimal=super_extra_minimal, convert_to_USDT=convert_to_USDT)

In [52]:
#from cryptocurrency.conversion_table import get_conversion_table
import time

t1 = time.time()
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, 
                                        offset_s=offset_s, dump_raw=False, as_pair=False, 
                                        minimal=False, extra_minimal=True, 
                                        super_extra_minimal=False, convert_to_USDT=False)
t2 = time.time()
print('Elapsed time:', t2 - t1)
conversion_table.sort_values(by='rolling_base_volume', ascending=False).head(50)

/tmp/ipykernel_4262/336667222.py:204: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  conversion_table_swapped.loc[:, ['symbol', 'price_change_percent', 'close', 'ask_price',
/tmp/ipykernel_4262/336667222.py:269: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  conversion_table_swapped.loc[:, ['open', 'close', 'bid_price', 'ask_price', 'USDT_open',
/tmp/ipykernel_4262/336667222.py:294: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only

Elapsed time: 9.32205057144165


/tmp/ipykernel_4262/336667222.py:436: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  conversion_table.loc[:, ['date', 'count']] = \


,base_asset,price_change_percent,bid_volume,ask_volume,bid_price,ask_price,count,rolling_base_volume,bid_ask_percent_change,bid_ask_volume_percent_change,close,rolling_quote_volume,symbol,quote_asset
date,,,,,,,,,,,,,,
2022-12-16 19:20:04.996,USDT,-0.077,1.410000e+02,1.445000e+03,15644.000000,15660.000000,16737.500000,7.090381e+09,0.102171,8.890290,698.710160,7.090381e+09,USDT,USDT
2022-12-16 19:20:05.114,BTC,-2.592,4.940000e-03,5.830000e-03,301894.000000,302225.000000,637444.384615,6.497751e+09,0.109521,45.868152,16332.086714,6.497751e+09,BTC,BTC
2022-12-16 19:20:05.081,BUSD,-0.585,6.473000e+02,2.420000e+01,1.019000,1.021000,34012.100000,4.219787e+09,0.195886,96.396128,438.319959,4.219787e+09,BUSD,BUSD
2022-12-16 19:20:05.060,ETH,-5.275,3.400000e-03,4.175000e-01,78211.600000,78233.400000,92235.500000,1.296944e+09,0.027865,0.807793,1166.616459,1.296944e+09,ETH,ETH
2022-12-16 19:20:05.088,BNB,-6.574,8.000000e-03,2.000000e-01,9635.000000,9655.000000,53124.538462,3.591818e+08,0.207147,3.846154,231.355539,3.591818e+08,BNB,BNB
2022-12-16 19:20:05.091,XRP,-4.183,4.292500e+03,3.031700e+03,5654.000000,5666.000000,24048.416667,1.730056e+08,0.211790,58.607083,0.376157,1.730056e+08,XRP,XRP
2022-12-16 19:20:04.824,DOGE,-6.005,1.603000e+04,2.700000e+03,0.428400,0.429100,35925.500000,1.566009e+08,0.163132,85.584624,209.758292,1.566009e+08,DOGE,DOGE
2022-12-16 19:20:04.732,EUR,-0.151,5.994600e+04,6.395350e+04,1.060500,1.060600,41364.000000,1.315423e+08,0.009429,48.382762,425.623355,1.315423e+08,EUR,EUR
2022-12-16 19:20:05.089,LTC,-6.481,7.048000e+00,6.252000e+00,4412.900000,4423.700000,29346.200000,9.862866e+07,0.244140,52.992481,67.650000,9.862866e+07,LTC,LTC


### Example testing the retrieval of connected assets directly (this is a big bottleneck!).

In [53]:
import time

def get_connected_assets(asset, exchange_info, priority='accuracy'):
    def reorder(connected_assets, priority):
        prioritized = [asset for asset in priority if asset in connected_assets]
        order = {asset: i for i, asset in enumerate(prioritized)}
        connected_assets_items = [asset for asset in connected_assets if asset in order]
        connected_assets_items.sort(key=order.get)
        connected_assets_iter = iter(connected_assets_items)
        return [next(connected_assets_iter) if asset in order 
                else asset for asset in connected_assets]
    priorities = {}
    priorities['accuracy'] = ['USDT', 'BTC', 'BUSD', 'ETH', 'BNB', 'AUD']
    priorities['fees'] = ['BUSD', 'BTC', 'BNB', 'ETH', 'USDT', 'AUD']
    priorities['wallet'] = ['BTC', 'ETH', 'BUSD', 'BNB', 'USDT', 'AUD']
    priority = priorities[priority]
    connected_base_assets = exchange_info['base_asset'] == asset
    connected_base_assets = exchange_info[connected_base_assets]
    connected_base_assets = connected_base_assets['quote_asset'].tolist()
    connected_quote_assets = exchange_info['quote_asset'] == asset
    connected_quote_assets = exchange_info[connected_quote_assets]
    connected_quote_assets = connected_quote_assets['base_asset'].tolist()
    connected_assets = sorted(list(set(connected_base_assets + connected_quote_assets)))
    return reorder(connected_assets, priority)

asset = 'AXS'

t1 = time.time()
connected_assets = get_connected_assets(asset, exchange_info, priority='fees')
t2 = time.time()
print('Elapsed time:', t2 - t1)
print(connected_assets)

Elapsed time: 0.0015759468078613281
['BUSD', 'BTC', 'BRL', 'BNB', 'ETH', 'USDT', 'TRY', 'AUD']


### Example testing convert_ohlcvs_from_pairs_to_assets (bootstrapping) directly.

In [ ]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion_table import get_conversion_table
import pandas as pd

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info

crypto_output_log_1min = 'crypto_logs/crypto_output_log_1min.txt'
df_1min = pd.read_csv(crypto_output_log_1min, header=[0, 1], index_col=0)
df_1min.index = pd.DatetimeIndex(df_1min.index)

conversion_table = df_1min.copy()
conversion_table.columns = conversion_table.columns.swaplevel(0, 1)
conversion_table = conversion_table.drop(columns=['rolling_base_volume', 
                                                  'rolling_quote_volume'])
conversion_table.columns = conversion_table.columns.swaplevel(0, 1)
conversion_table

In [ ]:
from cryptocurrency.conversion import convert_ohlcvs_from_pairs_to_assets

new_conversion_table = convert_ohlcvs_from_pairs_to_assets(conversion_table, 
                                                           exchange_info)
new_conversion_table

In [ ]:
new_conversion_table['ETH']

### Example testing the (as_pairs=True) bootstrapping of a minimal subset.

In [ ]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion import get_timezone_offset_in_seconds
from cryptocurrency.conversion_table import get_conversion_table, get_new_tickers
from cryptocurrency.conversion_ohlcv import convert_ohlcvs_from_pairs_to_assets
from cryptocurrency.ohlcvs import download_pairs
from cryptocurrency.resample import resample
from cryptocurrency.volume_conversion import add_rolling_volumes
from tqdm import tqdm
import pandas as pd
import os
import shutil

def bootstrap_loggers(client, assets, pairs=None, additional_intervals=None, upsampled_intervals=None, 
                      download_interval='1m', exchange_info=None, as_pair=False):
    log_file = 'crypto_logs/crypto_output_log_{}.txt'
    period = 2880 if download_interval == '1m' else 60
    second_period = 60 if download_interval == '1m' else None
    base_interval = download_interval + 'in' if download_interval[-1] == 'm' else download_interval
    frequency_1min = pd.tseries.frequencies.to_offset('1min')
    frequency_1d = pd.tseries.frequencies.to_offset('1d')
    frequency = pd.tseries.frequencies.to_offset(base_interval)
    pairs[base_interval] = download_pairs(client=client, assets=assets, interval=download_interval, 
                                          period=period, second_period=second_period)
    if not as_pair:
        pairs[base_interval] = convert_ohlcvs_from_pairs_to_assets(pairs[base_interval], exchange_info)
    pairs[base_interval] = add_rolling_volumes(pairs[base_interval])
    pairs[base_interval] = pairs[base_interval].loc[pairs[base_interval].dropna().first_valid_index():]
    if additional_intervals is not None:
        for additional_interval in tqdm(additional_intervals, unit=' pair'):
            pairs[additional_interval] = resample(pairs[base_interval].copy(), interval=additional_interval)
            pairs[additional_interval] = pairs[additional_interval].tail(200)
            pairs[additional_interval].to_csv(log_file.format(additional_interval))
    truncated_frequency = 60 if frequency > frequency_1min else 1500
    pairs[base_interval] = pairs[base_interval].tail(truncated_frequency)
    pairs[base_interval].to_csv(log_file.format(base_interval))
    if upsampled_intervals is not None:
        for subminute_interval in tqdm(upsampled_intervals, unit=' pair'):
            pairs[subminute_interval] = pairs[base_interval].tail(25)
            pairs[subminute_interval] = pairs[subminute_interval].resample(subminute_interval).agg('max')
            pairs[subminute_interval] = pairs[subminute_interval].fillna(method='pad').tail(60)
            pairs[subminute_interval].to_csv(log_file.format(subminute_interval))
    return pairs

as_pair = False
directory = 'crypto_logs'
if os.path.exists(directory):
    shutil.rmtree(directory)
os.mkdir(directory)
authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory=directory)
exchange_info = exchange.info
offset_s = get_timezone_offset_in_seconds()
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, 
                                        offset_s=offset_s, as_pair=True)
assets = get_new_tickers(conversion_table=conversion_table)
assets = assets[:10]

pairs = bootstrap_loggers(client=client, assets=assets, pairs={}, 
                          additional_intervals=['30min'], 
                          upsampled_intervals=['5s', '15s'], 
                          download_interval='1m', 
                          exchange_info=exchange_info, 
                          as_pair=True)
pairs['1min']